#Model Scoring

Using a scoring data set constructed in the `2_Feature_Engineering` notebook, and a model constructed in the `3_Model_Building_And_Training` notebook, this notebook loads the data and predicts the probability of component failure with the provided model. 

This notebook can be run though the `4_Scoring_Pipeline` notebook, which creates a temporary scoring data set before scoring the data with this notebook.

We provide the `4b_Model_Scoring_Evalation` notebook to examine the output of the scoring process.

**IMPORTANT NOTE** This notebook depends on there being a `scoring_data` set in the Databricks Data store. You can score any dataset constructed with the `2a_feature_engineering` notebook, but you must specify that data set in the `scoring_data` parameter above, or this notebook will fail. 

**Note:** This notebook should take less than a minute to execute all cells, depending on the compute configuration you have setup.

In [0]:
# import the libraries
from pyspark.ml import PipelineModel
# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

# The scoring uses the same feature engineering script used to train the model
scoring_table = 'testing_data'
results_table = 'results_output'

model = 'RandomForest' 

Databricks parameters to customize the runs.

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("scoring_data", scoring_table)
dbutils.widgets.text("results_data", results_table)

dbutils.widgets.text("model", model)

We need to run the feature engineering on the data we're interested in scoring (`2_Feature_Engineering`). Spark MLlib models require a vectorized data frame.<br> We transform the dataset here for model consumption. In a general scoring operation, we do not know the labels so we only need to construct the features.

In [0]:
sqlContext.refreshTable(dbutils.widgets.get("scoring_data")) 
spark.catalog.refreshTable(dbutils.widgets.get("scoring_data")) 
score_data = spark.table(dbutils.widgets.get("scoring_data"))

# We'll use the known label, and key variables.
label_var = ['label_e']
key_cols =['machineID','dt_truncated']

# Then get the remaining feature names from the data
input_features = score_data.columns

# We'll remove the known label, key variables and 
# a few extra columns we won't need.
remove_names = label_var + key_cols + ['failure','model_encoded','model' ]

# Remove the extra names that are in the input_features list
input_features = [x for x in input_features if x not in set(remove_names)]

input_features
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')

# assemble features
score_data = va.transform(score_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(score_data)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-163241296183660> in <module> 
 ----> 1 spark . catalog . refreshTable ( dbutils . widgets . get ( "scoring_data" ) ) 
 2 #sqlContext.refreshTable(dbutils.widgets.get("scoring_data")) 
 3 score_data = spark . table ( dbutils . widgets . get ( "scoring_data" ) ) 
 4 
 5 # We'll use the known label, and key variables. 

 /local_disk0/tmp/1603889809261-0/WidgetsHandlerImpl.py in get (self, name) 
 42 : return : Current value of the widget or default value
 43 """
 ---> 44 return self . _notebookArguments . getArgument ( name , self . _entry_point . getCurrentBindings ( ) ) 
 45 
 46 def getArgument ( self , name , defaultValue = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o221.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named scoring_data is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.GeneratedMethodAccessor589.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

To evaluate this model, we predict the component failures over the test data set. Since the test set has been created from data the model has not been seen before, it simulates future data. The evaluation then can be generalized to how the model could perform when operationalized and used to score the data in real time.

In [0]:
# Load the model from local storage
model_pipeline = PipelineModel.load("dbfs:/storage/models/" + dbutils.widgets.get("model") + ".pqt")

# score the data. The Pipeline does all the same operations on this dataset
predictions = model_pipeline.transform(score_data)

#write results to data store for persistance.
predictions.write.mode('overwrite').saveAsTable(dbutils.widgets.get("results_data"))

# Conclusion

An additional notebook (`4b_Model_Scoring_Evaluation`) has been included to examine how the process works.